# Memanggil Data

In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
file_path = '/content/drive/MyDrive/data_eda_step1.csv'
df = pd.read_csv(file_path)

In [ ]:
df.head(20000)

# Data Prepro

## Missing Value

In [ ]:
df.isnull().sum()

In [ ]:
df_filled = df.ffill().bfill()

Missing value diisi dengan nilai pada menit sebelumnya dan sesudahnya dengan asumsi bahwa data terekam secara kontinu, perubahan antar menit relatif stabil, serta tidak terdapat lonjakan nilai ekstrem dalam waktu singkat. Pendekatan ini sesuai untuk data sensor lingkungan yang cenderung memiliki pola halus dan bertahap.

In [ ]:
df_filled.head(100)

## Outlier

In [ ]:
df_filled['co2'].plot(figsize=(15, 4), title='CO2 Time Series')

In [ ]:
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers

# Contoh: deteksi outlier pada CO2
outlier_co2 = detect_outliers_iqr(df_filled, 'co2')
print(f"Jumlah outlier CO2: {len(outlier_co2)}")
outlier_co2.head()

In [ ]:
# Kalau timestamp masih kolom biasa
df_filled['timestamp'] = pd.to_datetime(df_filled['timestamp'])

# Cek apakah sudah urut naik
is_sorted = df_filled['timestamp'].is_monotonic_increasing
print(f"Timestamp sudah urut naik? {is_sorted}")

# Kalau belum, urutkan dulu
if not is_sorted:
    df = df_filled.sort_values(by='timestamp').reset_index(drop=True)
    print("Data sudah diurutkan berdasarkan timestamp.")


In [ ]:
import pandas as pd

# Pastikan timestamp sudah jadi datetime dan data sudah urut
df_filled['timestamp'] = pd.to_datetime(df_filled['timestamp'])
df_filled = df_filled.sort_values(by='timestamp').reset_index(drop=True)

# Set timestamp sebagai index supaya lebih mudah slicing
df_filled.set_index('timestamp', inplace=True)

# Timestamp outlier yang mau dicek
outlier_time = pd.to_datetime('2025-05-02 01:15:00')

# Data sebelum outlier: cari timestamp terbesar yang kurang dari outlier_time
before = df_filled[df_filled.index < outlier_time].iloc[-1]

# Data sesudah outlier: cari timestamp terkecil yang lebih dari outlier_time
after = df_filled[df_filled.index > outlier_time].iloc[0]

# Data outlier (kalau ada persis)
outlier = df_filled.loc[df_filled.index == outlier_time]

print("Data 1 menit sebelum outlier:")
print(before)

print("\nData outlier:")
print(outlier if not outlier.empty else "Data outlier tidak ditemukan persis.")

print("\nData 1 menit sesudah outlier:")
print(after)


Kenaikan CO2 tersebut wajar karena terjadi secara bertahap dan berurutan, tidak ada lonjakan tiba-tiba yang ekstrem. Jadi nilai yang tampak sebagai outlier sebenarnya merupakan bagian dari tren alami atau fluktuasi harian data.

## Normalisasi

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
cols_to_scale = ['co2', 'temperature', 'humidity', 'rainfall', 'pyrano']

df_scaled = df_filled.copy()
df_scaled[cols_to_scale] = scaler.fit_transform(df_scaled[cols_to_scale])

# Jangan lupa reset index jika sebelumnya pakai timestamp sebagai index
df_scaled = df_scaled.reset_index()

print(df_scaled.head())

In [ ]:
df_scaled.to_csv('/content/drive/MyDrive/df_scaled.csv', index=False)